In [4]:
#!/usr/bin/env python3
# code to replicate the performance of the NIPS'16 MLHC workshop paper "Cam CNNS predict anatomy?"

from __future__ import absolute_import
from __future__ import print_function

import os
import csv
import six

import numpy as np
import time
import json
import warnings

from collections import deque
from collections import OrderedDict
from collections import Iterable

# os.environ["THEANO_FLAGS"] = "device=gpu0,floatX=float32" 

os.environ['KERAS_BACKEND'] = 'tensorflow'
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu,floatX=float32,force_device=true"#,lib.cnmem=0.7 ,nvcc.flags=-D_FORCE_INLINES"
import sys
import numpy as np
from keras import backend as K
K.set_image_data_format('channels_first')
#from keras.datasets import cifar10
#keras.callbacks.Callback()
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Convolution2D, MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback
from keras.models import load_model
from keras.datasets import mnist
from scipy import io
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
# from Ipython.display import display, clear_output


In [6]:
def load_data():
    X = np.load('/work/vsankar/Project-Luna/image_array.npy')
    Y = np.load('/work/vsankar/Project-Luna/new_labels.npy')
    Tracer()()
    x_images = np.empty([1,1,X.shape[2],X.shape[3]])
    y_labels = np.empty([1])
#     for i,y in enumerate(Y):
#         Tracer()()
#         if y > 4.0:
#             Y[i] = 4.0
    n=0;
    for i,y in enumerate(Y):
        print(i)
        sys.stdout.flush()
        if y<=2.0:
            Y[i] = 0.0
#             y_labels = np.append(y_labels,[0.0],axis=0)
#             x_images = np.append(x_images,X[i:i+1,:,:,:],axis=0)
            
        elif y>=4:
            Y[i] = 1.0
#             y_labels = np.append(y_labels,[1.0],axis=0)
#             x_images = np.append(x_images,X[i:i+1,:,:,:],axis=0)
            
            

    a = range(len(y_labels))
    np.random.shuffle(a)
    idx_test = a[:2510]
    idx_train = a[2510:]
    
#     Tracer()()
    
    X_train = x_images[idx_train]
    X_test = x_images[idx_test]
    y_train = y_labels[idx_train]
    y_test = y_labels[idx_test]

    return X_train,y_train,X_test,y_test


In [7]:
def normalize_date(X_train,Y_train,X_test,Y_test,nb_classes):
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    Y_train = Y_train.astype('float32')
    Y_test = Y_test.astype('float32')

#     normalizing the data
    X_train /= 4095.0   
    X_test /= 4095.0
    
    #std
#     X_train = X_train/np.std(X_train) - np.mean(X_train)
#     X_test = X_test/np.std(X_test) - np.mean(X_test)
    
#     Tracer()()
    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(Y_train, nb_classes)
    Y_test = np_utils.to_categorical(Y_test, nb_classes)
    
    return X_train,Y_train,X_test,Y_test

In [8]:
X_test,y_test,X_train,y_train = load_data()
nb_classe=2
X_train,Y_train,X_test,Y_test = normalize_date(X_train,y_train,X_test,y_test,nb_classe)

/home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: `Tracer` is deprecated since version 5.1, directly use `IPython.core.debugger.Pdb.set_trace()`


> <ipython-input-6-a7fe9aff159c>(5)load_data()
      3     Y = np.load('/work/vsankar/Project-Luna/new_labels.npy')
      4     Tracer()()
----> 5     x_images = np.empty([1,1,X.shape[2],X.shape[3]])
      6     y_labels = np.empty([1])
      7 #     for i,y in enumerate(Y):

ipdb> q
Exiting Debugger.
